In [2]:
# ========= 1) Load & light clean =========
import re
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

# 路径设置
data_path   = r"C:/Users/ZITIAO/Desktop/merged_for_regression.csv"
export_path = r"C:/Users/ZITIAO/Desktop/industry_regression_summary.csv"

# 读取
df = pd.read_csv(data_path)

# 统一列名：去两端空格，连着空格换成单空格，统一为小写比较再找列（不改变原始可视列名）
norm = {c: re.sub(r'\s+', ' ', str(c)).strip() for c in df.columns}
df.rename(columns=norm, inplace=True)

# 一个小工具：模糊查找列名（忽略大小写、空格）
def find_col(cands):
    cands = [re.sub(r'\s+',' ', s).strip().lower() for s in cands]
    for c in df.columns:
        if re.sub(r'\s+',' ', c).strip().lower() in cands:
            return c
    return None

# 必要列定位
col_emis  = find_col(['total emission (kg)'])
col_emp   = find_col(['employee'])
col_turnM = find_col(['turnover_million'])   # 我们优先用“百万”单位
col_turn  = find_col(['turnover'])

# 行业列定位（兼容两种命名）
col_sector    = find_col(['sector','regulated industry sector'])
col_subsector = find_col(['subsector','regulated industry sub sector'])

# 保底校验
need = {'Total Emission (kg)':col_emis, 'Employee':col_emp}
if not col_turnM and not col_turn:
    raise KeyError("找不到 Turnover 或 Turnover_million 任一列。")
for k,v in need.items():
    if v is None:
        raise KeyError(f"缺列：{k}")

# 选择 Turnover 列（优先 _million）
col_turn_used = col_turnM if col_turnM else col_turn

# 行业列：如果存在则转为类别并规范成统一新名，便于公式引用
if col_sector:
    df['Sector'] = df[col_sector].astype('category')
if col_subsector:
    df['SubSector'] = df[col_subsector].astype('category')

# ========= 2) Make logs (+ keep only valid rows) =========
# 数值化 + 正数过滤 + log
def to_log_pos(s):
    s_num = pd.to_numeric(s, errors='coerce')
    s_num = s_num.where(s_num > 0)   # 非正置 NaN
    return np.log(s_num)

df['log_emission'] = to_log_pos(df[col_emis])
df['log_employee'] = to_log_pos(df[col_emp])
df['log_turnover'] = to_log_pos(df[col_turn_used])

# 只在需要变量上做完整行
needed_vars = ['log_emission','log_turnover','log_employee']
if 'Sector' in df.columns:    needed_vars.append('Sector')
if 'SubSector' in df.columns: needed_vars.append('SubSector')

clean = df.replace([np.inf,-np.inf], np.nan).dropna(subset=needed_vars)

print(f"样本量（完整行）: {len(clean)}")
print(clean[needed_vars].head(3))


样本量（完整行）: 2015
   log_emission  log_turnover  log_employee          Sector       SubSector
0      0.139762      7.077498      8.396606  Water Industry  Water Industry
1      0.139762      7.077498      8.396606  Water Industry  Water Industry
2      0.139762      7.077498      8.396606  Water Industry  Water Industry


In [13]:
# ========= 3) Define and run the 4 regressions =========

# 构造四个公式（按你的要求）
formulas = {
    'Base'      : 'log_emission ~ log_turnover + log_employee',
    'Sector'    : 'log_emission ~ log_turnover + log_employee + C(Sector)',
    'SubSector' : 'log_emission ~ log_turnover + log_employee + C(SubSector)',
    'Both'      : 'log_emission ~ log_turnover + log_employee + C(Sector) + C(SubSector)'
}

# 根据数据里是否存在 Sector/SubSector，自动筛掉不可用模型
usable = {}
for name, fml in formulas.items():
    if ('C(Sector)' in fml and 'Sector' not in clean.columns): 
        continue
    if ('C(SubSector)' in fml and 'SubSector' not in clean.columns):
        continue
    usable[name] = fml

print("Estimate model：", list(usable.keys()))

# 运行并打印 summary，同时收集系数到汇总表
rows = []
for name, fml in usable.items():
    print("\n" + "="*10, f"{name} Regression", "="*10)
    model = smf.ols(fml, data=clean).fit()
    print(model.summary())

    # 收集系数
    coefs = model.params
    ses   = model.bse
    tvals = model.tvalues
    pvals = model.pvalues
    for term in coefs.index:
        rows.append({
            'model': name,
            'term': term,
            'coef': coefs[term],
            'std_err': ses[term],
            't': tvals[term],
            'p': pvals[term],
            'nobs': model.nobs,
            'r2': model.rsquared
        })

summary_df = pd.DataFrame(rows)
summary_df.to_csv(export_path, index=False)
#print(f"\n已导出汇总表：{export_path}")
summary_df.head(10)


Estimate model： ['Base', 'Sector', 'SubSector', 'Both']

========== Base Regression ==========
                            OLS Regression Results                            
Dep. Variable:           log_emission   R-squared:                       0.335
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     507.9
Date:                Tue, 12 Aug 2025   Prob (F-statistic):          2.77e-179
Time:                        16:55:50   Log-Likelihood:                -4941.1
No. Observations:                2015   AIC:                             9888.
Df Residuals:                    2012   BIC:                             9905.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

,model,term,coef,std_err,t,p,nobs,r2
0,Base,Intercept,-5.369199,0.347427,-15.454172,5.347061e-51,2015.0,0.335482
1,Base,log_turnover,0.802672,0.072113,11.130788,5.719829e-28,2015.0,0.335482
2,Base,log_employee,0.397993,0.088928,4.475468,8.050551e-06,2015.0,0.335482
3,Sector,Intercept,-1.821720,0.773417,-2.355416,1.859777e-02,2015.0,0.424588
4,Sector,C(Sector)[T.Landfill],2.250201,0.591852,3.801963,1.478415e-04,2015.0,0.424588
5,Sector,C(Sector)[T.Metals],-0.095074,0.551959,-0.172248,8.632598e-01,2015.0,0.424588
6,Sector,C(Sector)[T.Paper and textiles],0.009727,0.543341,0.017901,9.857193e-01,2015.0,0.424588
7,Sector,C(Sector)[T.Refineries & Fuel],-4.048834,0.417877,-9.689054,9.962225e-22,2015.0,0.424588
8,Sector,C(Sector)[T.Waste treatment],4.397909,0.586725,7.495684,9.838357e-14,2015.0,0.424588
9,Sector,C(Sector)[T.Water Industry],1.578745,0.453047,3.484724,5.032240e-04,2015.0,0.424588


In [4]:
# ========= 4) （可选）快速对比四个模型核心变量 =========
# 看看 log_turnover / log_employee 在不同模型下的稳定性
pivot = (summary_df
         .query("term in ['log_turnover','log_employee']")
         .pivot(index='term', columns='model', values='coef'))
print("\n关键变量各模型系数对比：")
print(pivot.round(4))



关键变量各模型系数对比：
model           Base    Both  Sector  SubSector
term                                           
log_employee  0.3980 -1.1642 -1.1784    -1.1642
log_turnover  0.8027  1.8801  1.9375     1.8801


In [5]:
# ========= 5) （可选）把四个模型的核心指标单独导出 =========
brief = (summary_df
         .query("term in ['Intercept','log_turnover','log_employee']")
         .copy())
brief.to_csv(r"C:/Users/ZITIAO/Desktop/industry_regression_brief.csv", index=False)
print("已导出简表：industry_regression_brief.csv")


已导出简表：industry_regression_brief.csv


In [15]:
# =========================
# Panel OLS on existing `df` (firm × year)
# - Model A: Two-way FE (firm + year)
# - Model B: Sector dummies + Year FE (no firm FE)
# - Model C: SubSector dummies + Year FE (no firm FE)
# =========================
import re
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS

# ---- 0) 安全检查 ----
if 'df' not in globals():
    raise RuntimeError("当前环境未发现 DataFrame `df`。请先在上一个代码框创建/读取 df。")

# 不修改原 df
src = df.copy()
src.columns = [re.sub(r'\s+', ' ', str(c)).strip() for c in src.columns]

def find_col(name_options):
    """大小写/多空格不敏感地匹配列名，返回真实列名或 None"""
    canon = {re.sub(r'\s+',' ', c).strip().lower(): c for c in src.columns}
    for opt in name_options:
        key = re.sub(r'\s+',' ', opt).strip().lower()
        if key in canon: 
            return canon[key]
    return None

# ---- 1) 解析关键列 ----
col_firm   = find_col(["OPERATOR NAME","operator name","company","company name"])
col_year   = find_col(["Year","year"])
col_emis   = find_col(["Total Emission (kg)","total emission (kg)","emission"])
col_emp    = find_col(["Employee","Employees"])
col_turn_m = find_col(["Turnover_million","turnover_million"])
col_turn   = find_col(["Turnover","turnover"])
col_sector = find_col(["REGULATED INDUSTRY SECTOR","Sector","sector"])
col_subsec = find_col(["REGULATED INDUSTRY SUB SECTOR","SubSector","subsector"])

if not all([col_firm, col_year, col_emis, col_emp]) or (not col_turn_m and not col_turn):
    raise KeyError(f"缺少关键列，请检查：firm={col_firm}, year={col_year}, emission={col_emis}, "
                   f"employee={col_emp}, turnover={col_turn_m or col_turn}")

turn_used = col_turn_m if col_turn_m else col_turn

# ---- 2) 聚合为 公司×年份 （排放求和；员工/营业额取当年首个非空；行业取当年首个非空）----
def first_valid(s):
    s = s.dropna()
    return s.iloc[0] if len(s) else np.nan

agg = (src
       .groupby([col_firm, col_year], as_index=False)
       .agg({
           col_emis: 'sum',
           col_emp: first_valid,
           turn_used: first_valid,
           **({col_sector: first_valid} if col_sector else {}),
           **({col_subsec: first_valid} if col_subsec else {}),
       }))

# ---- 3) 生成对数变量（>0 才能取对数）----
def safe_log(x):
    x = pd.to_numeric(x, errors="coerce")
    x = x.where(x > 0)
    return np.log(x)

agg = agg.rename(columns={col_firm:'firm', col_year:'year',
                          col_emis:'emission_kg', col_emp:'employee',
                          turn_used:'turnover_used',
                          **({col_sector:'Sector'} if col_sector else {}),
                          **({col_subsec:'SubSector'} if col_subsec else {})})
# 年份转为整数（若可能）
with np.errstate(all='ignore'):
    agg['year'] = pd.to_numeric(agg['year'], errors='coerce').astype('Int64')

agg['log_emission'] = safe_log(agg['emission_kg'])
agg['log_employee'] = safe_log(agg['employee'])
agg['log_turnover'] = safe_log(agg['turnover_used'])

# 基础清洗
keep_cols = ['firm','year','log_emission','log_employee','log_turnover']
if 'Sector' in agg.columns:    keep_cols.append('Sector')
if 'SubSector' in agg.columns: keep_cols.append('SubSector')

panel = (agg[keep_cols]
         .replace([np.inf, -np.inf], np.nan)
         .dropna(subset=['firm','year','log_emission','log_employee','log_turnover']))

# 类别化
if 'Sector' in panel.columns:    panel['Sector'] = panel['Sector'].astype('category')
if 'SubSector' in panel.columns: panel['SubSector'] = panel['SubSector'].astype('category')

# 设置面板索引
panel = panel.set_index(['firm','year']).sort_index()

print(f"Panel shape: {panel.shape}, firms={panel.index.levels[0].size}, years≈{panel.index.levels[1].size}")
print(panel[['log_emission','log_turnover','log_employee']].head())

# ---- 4) Model A: 双固定效应（公司 + 年份）----
mod_A = PanelOLS.from_formula(
    "log_emission ~ 1 + log_turnover + log_employee + EntityEffects + TimeEffects",
    data=panel
)
res_A = mod_A.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
print("\n" + "="*80)
print("Model A — Two-way FE (firm + year)")
print("="*80)
print(res_A.summary)

# ---- 5) Model B: Sector dummies + 年份固定效应（不加公司FE，否则 Sector 被吸收）----
res_B = None
if 'Sector' in panel.columns:
    d_sec = pd.get_dummies(panel['Sector'], prefix='Sector', drop_first=True)
    X_B = pd.concat([panel[['log_turnover','log_employee']], d_sec], axis=1)
    data_B = pd.concat([panel[['log_emission']], X_B], axis=1).dropna()
    mod_B = PanelOLS(
        dependent=data_B['log_emission'],
        exog=data_B.drop(columns=['log_emission']),
        time_effects=True
    )
    res_B = mod_B.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
    print("\n" + "="*80)
    print("Model B — Sector dummies + Year FE (no firm FE)")
    print("="*80)
    print(res_B.summary)
else:
    print("\n[提示] 未找到 Sector 列，跳过 Model B。")

# ---- 6) Model C: SubSector dummies + 年份固定效应（不加公司FE）----
res_C = None
if 'SubSector' in panel.columns:
    d_sub = pd.get_dummies(panel['SubSector'], prefix='SubSector', drop_first=True)
    X_C = pd.concat([panel[['log_turnover','log_employee']], d_sub], axis=1)
    data_C = pd.concat([panel[['log_emission']], X_C], axis=1).dropna()
    mod_C = PanelOLS(
        dependent=data_C['log_emission'],
        exog=data_C.drop(columns=['log_emission']),
        time_effects=True
    )
    res_C = mod_C.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
    print("\n" + "="*80)
    print("Model C — SubSector dummies + Year FE (no firm FE)")
    print("="*80)
    print(res_C.summary)
else:
    print("\n[提示] 未找到 SubSector 列，跳过 Model C。")

# ---- 7) 汇总关键结果（可选导出）----
rows = []
def collect(res, name):
    if res is None: return
    params = res.params
    pvals  = res.pvalues
    tvals  = res.tstats
    rsq_w  = getattr(res.rsquared, 'within',  np.nan)
    rsq_o  = getattr(res.rsquared, 'overall', np.nan)
    for k in params.index:
        rows.append({
            'model': name, 'term': k,
            'coef': params[k],
            't': tvals.get(k, np.nan),
            'p': pvals.get(k, np.nan),
            'R2_within': rsq_w, 'R2_overall': rsq_o,
            'nobs': res.nobs
        })

collect(res_A, "Two-way FE")
collect(res_B, "Sector + Year FE")
collect(res_C, "SubSector + Year FE")

summary = pd.DataFrame(rows)
print("\nTidy summary (head):")
print(summary.head(10))

# 如需导出，取消下行注释并修改路径
summary.to_csv(r"C:/Users/ZITIAO/Desktop/panel_ols_summary.csv", index=False)


Panel shape: (125, 5), firms=26, years≈8
                    log_emission  log_turnover  log_employee
firm          year                                          
anglian water 2016      9.506717      7.077498      8.396606
              2017      9.251789      7.112327      8.403352
              2018      9.020438      7.130018      8.433812
              2019      9.024830      7.211557      8.468843
              2020      9.177222      7.258412      8.513988

Model A — Two-way FE (firm + year)
                          PanelOLS Estimation Summary                           
Dep. Variable:           log_emission   R-squared:                        0.0375
Estimator:                   PanelOLS   R-squared (Between):              0.5359
No. Observations:                 125   R-squared (Within):               0.0133
Date:                Tue, Aug 12 2025   R-squared (Overall):              0.5645
Time:                        16:57:35   Log-likelihood                   -119.43
Cov. Estim